In [1]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [2]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

f:\Desktop\reddit-news-bot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = Ollama(model="gemma:7b", temperature=0, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

# Prompt
template = """Use the following pieces of context to answer the question at the end. 
The context is a collection of news from last week in the form of posts structured in JSON format with the following schema:
    date: the date in which the post was made, formatted as day-month-year
    title: the post title, which usually contains a brief summary of the topic
    body: the post body, when present it contains the main information of the post
    upvote_ratio: a representation of the post reception on a scale of 0 to 1
    subreddit: the subreddit the post is from
    num_comments: how many comments the post has
    submission_flair: the post category, ignore meme posts when providing news
    has_image: wether the post contains an image or not
    url: the url of the post or of its contents
    comments: a list of comments left by users, adds info but it's much less important
---
Context: 
{context}
---
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [4]:
result = qa_chain({"query": "Based on the context, What are the most interesting developments related to Large Language Models, Image Generation, and Ai during the past week?"})

f:\Desktop\reddit-news-bot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, "Connessione in corso interrotta forzatamente dall'host remoto", None, 10054, None))